# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [ ]:
# Load env variable

%load_ext dotenv
%dotenv 

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [14]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
# Get Dask
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [ ]:
import os
from glob import glob

price_data_directory='../../05_src/data/prices' # Loading PRICE_DATA 
PRICE_DATA = os.getenv('PRICE_DATA')

# Use glob to find all parquet files
parquet_files = glob(os.path.join(price_data_directory, '**/*.0.parquet'),recursive=True)
print("Number of Parquet files:", len(parquet_files))

Number of Parquet files: 11207


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [15]:
dd_px = dd.read_parquet(PRICE_DATA).set_index("ticker") # Loading parquet files
print("Before Transformation, dataset is:", dd_px.head(5))

# Transformation
dd_features = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
).assign(
    positive_return = lambda x: (x['returns'] > 0)*1
).assign(
    hi_lo_range = lambda x: x['High'] - x['Low']
))

print("After Transformation, dataset is:", dd_features.head(5))

Before Transformation, dataset is:              Date       Open       High        Low      Close  Adj Close  \
ticker                                                                     
A      2000-01-03  56.330471  56.464592  48.193848  51.502148  43.463017   
A      2000-01-04  48.730328  49.266811  46.316166  47.567955  40.142952   
A      2000-01-05  47.389126  47.567955  43.141991  44.617310  37.652863   
A      2000-01-06  44.080830  44.349072  41.577251  42.918453  36.219193   
A      2000-01-07  42.247852  47.165592  42.203148  46.494991  39.237457   

         Volume       sector                       subsector  year  
ticker                                                              
A       4674353  Health Care  Life Sciences Tools & Services  2000  
A       4765083  Health Care  Life Sciences Tools & Services  2000  
A       5758642  Health Care  Life Sciences Tools & Services  2000  
A       2534434  Health Care  Life Sciences Tools & Services  2000  
A       2819626  H

+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [16]:
# Convert transformed features set to pandas
dd_pd = dd_features.compute()

# Calculating rolling average of 10 days
dd_pd = (dd_pd.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(rolling_avg_return = x['returns'].rolling(10).mean())
))

print(dd_pd.shape)
print("After Rolling Average of 10 days, the dataset is:", dd_pd.head(5))



(2780684, 15)
After Rolling Average of 10 days, the dataset is:              Date        Open        High         Low       Close   Adj Close  \
ticker                                                                          
HUM    2019-01-02  283.309998  283.859985  277.230011  281.589996  269.714508   
HUM    2019-01-03  281.399994  283.019989  269.179993  269.700012  258.325958   
HUM    2019-01-04  272.309998  279.720001  271.070007  276.619995  264.954071   
HUM    2019-01-07  277.000000  279.880005  274.529999  276.579987  264.915802   
HUM    2019-01-08  278.709991  280.299988  271.209991  274.929993  263.335388   

         Volume       sector            subsector  year  Close_lag_1  \
ticker                                                                 
HUM     1168800  Health Care  Managed Health Care  2019          NaN   
HUM     1330600  Health Care  Managed Health Care  2019   281.589996   
HUM     1698900  Health Care  Managed Health Care  2019   269.700012   
HUM     

In [24]:
# Different datatypes we used for this dataset

print("Actual Parquet files:", type(dd_px))
print("After Transformation, feature dataset:", type(dd_features))
print("After converting to Pandas:", type(dd_pd))

Actual Parquet files: <class 'dask_expr._collection.DataFrame'>
After Transformation, feature dataset: <class 'dask_expr._collection.DataFrame'>
After converting to Pandas: <class 'pandas.core.frame.DataFrame'>


Please comment:

+ **Was it necessary to convert to pandas to calculate the moving average return?**
No, converting to pandas wasn’t required for calculating the moving average. Dask can do this directly, which is better for large datasets. But if the data is small, using pandas can be easier.



+ **Would it have been better to do it in Dask? Why?**
Yes, doing it in Dask would be better for large datasets. Dask handles big data by working in smaller chunks, so it’s faster and avoids memory issues.


(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.